In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110550

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,52884.9910
2018-02-28,43180.1971
2018-03-31,54990.8941
2018-04-30,45621.8610
2018-05-31,55028.4709
2018-06-30,55994.2801
2018-07-31,55068.6007
2018-08-31,57106.9086
2018-09-30,73335.8001


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    52884.9910
 2018-02-28    43180.1971
 2018-03-31    54990.8941
 2018-04-30    45621.8610
 2018-05-31    55028.4709
 2018-06-30    55994.2801
 2018-07-31    55068.6007
 2018-08-31    57106.9086
 2018-09-30    73335.8001
 2018-10-31    60765.4425
 2018-11-30    58119.0022
 2018-12-31    59306.3457
 2019-01-31    72173.8093
 2019-02-28    58502.2662
 2019-03-31    69327.7183
 2019-04-30    66302.6544
 2019-05-31    76008.4819
 2019-06-30    78625.1308
 2019-07-31    85762.9905
 2019-08-31    87247.8933
 2019-09-30    64457.9060
 2019-10-31    65477.8507
 2019-11-30    72290.8004
 2019-12-31    73331.1493
 2020-01-31    59472.1146
 2020-02-29    59949.4080
 2020-03-31    52472.2741
 2020-04-30    60589.1666
 2020-05-31    62854.4254
 2020-06-30    65122.3526
 2020-07-31    70119.8399
 2020-08-31    63016.5227
 2020-09-30    61839.2171
 2020-10-31    62790.5208
 2020-11-30    57276.5750
 2020-12-31    57631.4209
 2021-01-31    54015.9290
 2021-02-

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.185564
p-value : 0.020836
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=889.316, Time=0.02 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=880.370, Time=0.12 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=876.727, Time=0.06 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=878.395, Time=0.06 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=878.090, Time=0.08 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=inf, Time=0.37 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=872.364, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=873.755, Time=0.04 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=875.392, Time=0.07 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=877.095, Time=0.09 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=876.832, Time=0.08 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=872.903, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=874.478, Time=0.10 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=875.444, Time=0.18 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=877.083, Time=0.20 sec
 ARIMA(3,0,0)(0,0,0)[1] inter

ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(1,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(1, 0)   Log Likelihood                -433.179
Method:                       css-mle   S.D. of innovations           7255.039
Date:                Sat, 11 Sep 2021   AIC                            872.358
Time:                        04:03:57   BIC                            877.571
Sample:                             0   HQIC                           874.269
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        6.23e+04   2737.038     22.761      0.000    5.69e+04    6.77e+04
ar.L1.y        0.6037      0.121      4.975      0.000       0.366       0.842
                                    Roots           

# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([60672.97747039, 61316.6779745 ]), array([7255.03855274, 8474.65173232]), array([[46453.36320057, 74892.59174021],
       [44706.66579762, 77926.69015138]]))
